In [1]:
from weavepy import *
import xarray as xr
import seaborn as sns
sns.set_context("talk")

In [7]:
# --- Parameters for loading data ---
# It is expected that these will be changed rarely
## Variables
climate_variable = "TA"
energy_variable = "SPV"
vars = [climate_variable, energy_variable]
techno = "60" # Techno needs to be specified for energy variables
## Temporal aggregation
aggregation_frequency = "D"
aggregation_function = "mean"
## Selection
country = "AL"
models = []
scenarios = []

In [8]:
data = load_vars([climate_variable, energy_variable], bdd_version = BDD_VERSION,
              countries = [country], technos = ["NA", techno], models = models, scenarios = scenarios, 
              aggregation_frequency = aggregation_frequency, aggregation_function = aggregation_function,
              verbose=True)

Loading country 1/1 (AL)
|_Variable 1/2 (TA)
|__Techno 1/1 (NA)
|__ > File exists, loading from cache: /Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/PECD4.2/TA_NA_D-mean_AL_PECD4.2.nc
|_Variable 2/2 (SPV)
|__Techno 1/1 (60)
|__ > File not found, loading from source. Estimated processing time = 1 minute.


FileNotFoundError: No matching CSV files found for variable 'SPV' in /Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/PECD4.2

In [4]:
# --- Period ---
# These can be changed and adjusted easily
historical_period = (1990, 2020)
future_period = (2070, 2100)

In [5]:
# --- Thresholds ---
# These can be changed and adjusted easily
comparisons = { # For each variable, choose among '<', '<=', '>', or '>='
    climate_variable:'<=', 
    energy_variable:'<',
}
thresholds = { # Number in units of variable
    climate_variable: 0,
    energy_variable: 0.30
}

In [6]:
# Identify problematic days
days = {}
for var in vars:
    days[var] = identify_pb_days(data[var], comparisons[var], thresholds[var])
days["compound"] = xr.concat(list(days.values()), dim = "var").all(dim = "var").rename("compound")
# Identify events
events = {}
for var in vars + ["compound"]:
    events[var] = identify_events_whole_base(days[var])

KeyError: "No variable named 'WON'. Variables on the dataset include ['time', 'scenario', 'model', 'TA', 'country', 'tech']"

## Number of events

In [ ]:
event_count_barplot_multi(list(events.values()), historical_period, future_period)

In [ ]:
nb_event_timeseries_multi(list(events.values()))

## Duration 

In [ ]:
event_duration_hist_multi(list(events.values()), historical_period, future_period)

## Seasonality

In [ ]:
event_seasonality_kde_multi(list(days.values()), historical_period, future_period)